In [1]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import torch
import random
from copy import deepcopy
import gc
import os
from datetime import datetime
import json
import stanza
from nltk.tree import Tree
from dep_func import depTripleFuncLex

c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
nlp_dependency = stanza.Pipeline('zh')

2024-09-16 13:58:36 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-09-16 13:58:37 INFO: Downloaded file to C:\Users\Administrator\stanza_resources\resources.json
2024-09-16 13:58:37 INFO: "zh" is an alias for "zh-hans"
2024-09-16 13:58:38 INFO: Loading these models for language: zh-hans (Simplified_Chinese):
| Processor    | Package          |
-----------------------------------
| tokenize     | gsdsimp          |
| pos          | gsdsimp_charlm   |
| lemma        | gsdsimp_nocharlm |
| constituency | ctb-51_charlm    |
| depparse     | gsdsimp_charlm   |
| sentiment    | ren_charlm       |
| ner          | ontonotes        |

2024-09-16 13:58:38 INFO: Using device: cuda
2024-09-16 13:58:38 INFO: Loading: tokenize
2024-09-16 13:58:38 INFO: Loading: pos
2024-09-16 13:58:39 INFO: Loading: lemma
2024-09-16 13:58:39 INFO: Loading: constituency
2024-09-16 13:58:39 INFO: Loading: depparse
2024-09-16 13:58:39 INFO: Loading: sentiment
2024-09-16 13:58:40 INFO: Loading: ner
2024-09-16 13:58:41 INFO: Done loading processors!


In [3]:
def get_constituency_parsing(sentence):
    doc = nlp_dependency(sentence)
    sentence_constituency = str(doc.sentences[0].constituency)
    
    return sentence_constituency


#1.0 NP→PN
def NP_PN(t,vector):
    if t.label() == 'NP' and len(t) >= 1:
        if t[0].label() == 'PN':
            vector[0] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            NP_PN(subtree,vector)

# 4.0 NP→DP NP
def NP_DP_NP(t,vector):
    if t.label() == 'NP' and len(t) >= 2:
        if t[0].label() == 'DP' and t[1].label() == 'NP':
            vector[1] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            NP_DP_NP(subtree,vector)

# 6.2 DP→DT
def DP_DT(t,vector):
    if t.label() == 'DP' and len(t) >= 1:
        if t[0].label() == 'DT':
            vector[2] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            DP_DT(subtree,vector)

#6.6 IP→NP VP PU
def IP_NP_VP_PU(t,vector):
    if t.label() == 'IP' and len(t) >= 3:
        if t[0].label() == 'NP' and t[1].label() == 'VP' and t[2].label() == 'PU':
            vector[3] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            IP_NP_VP_PU(subtree,vector)

#6.8 PRN→PU NP PU
def PRN_PU_NP_PU(t,vector):
    if t.label() == 'PRN' and len(t) >= 3:
        if t[0].label() == 'PU' and t[1].label() == 'NP' and t[2].label() == 'PU':
            vector[4] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            PRN_PU_NP_PU(subtree,vector)

#6.8 NP→NR
def NP_NR(t,vector):
    if t.label() == 'NP' and len(t) >= 1:
        if t[0].label() == 'NR':
            vector[5] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            NP_NR(subtree,vector)

#10.0 CP_ADVP_IP
def CP_ADVP_IP(t,vector):
    if t.label() == 'CP' and len(t) >= 2:
        if t[0].label() == 'ADVP' and t[1].label() == 'IP':
            vector[6] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            CP_ADVP_IP(subtree,vector)

#10.6 NP_DNP_NP
def NP_DNP_NP(t,vector):
    if t.label() == 'NP' and len(t) >= 2:
        if t[0].label() == 'DNP' and t[1].label() == 'NP':
            vector[7] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            NP_DNP_NP(subtree,vector)
            
#16.4 ADVP_CS
def ADVP_CS(t,vector):
    if t.label() == 'ADVP' and len(t) >= 1:
        if t[0].label() == 'CS':
            vector[8] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            ADVP_CS(subtree,vector)

#16.8 DNP_NP_DEG
def DNP_NP_DEG(t,vector):
    if t.label() == 'DNP' and len(t) >= 2:
        if t[0].label() == 'NP' and t[1].label() == 'DEG': 
            vector[9] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            DNP_NP_DEG(subtree,vector)

def NP_QP_DNP_NP(t,vector):
    if t.label() == 'NP' and len(t) >= 3:
        if t[0].label() == 'QP' and t[1].label() == 'DNP' and t[2].label() == 'NP':
            vector[10] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            NP_QP_DNP_NP(subtree,vector)

def NP_NP_PRN(t,vector):
    if t.label() == 'NP' and len(t) >= 2:
        if t[0].label() == 'NP' and t[1].label() == 'PRN':
            vector[11] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            NP_NP_PRN(subtree,vector)

def NP_NR_CC_NR(t,vector):
    if t.label() == 'NP' and len(t) >= 3:
        if t[0].label() == 'NR' and t[1].label() == 'CC' and t[2].label() == 'NR':
            vector[12] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            NP_NR_CC_NR(subtree,vector)

def NP_NP_CC_NP(t,vector):
    global score, count
    if t.label() == 'NP' and len(t) >= 3:
        if t[0].label() == 'NP' and t[1].label() == 'CC' and t[2].label() == 'NP':
            vector[13] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            NP_NP_CC_NP(subtree,vector)
            
def get_feature_vector(sentence,vector):
    
    sentence_constituency = get_constituency_parsing(sentence)
    #CFGR features
    tree = Tree.fromstring(sentence_constituency)
    find_NP_PN = NP_PN(tree,vector)
    find_NP_DP_NP = NP_DP_NP(tree,vector)
    find_DP_DT = DP_DT(tree,vector)
    find_IP_NP_VP_PU = IP_NP_VP_PU(tree,vector)
    find_PRN_PU_NP_PU = PRN_PU_NP_PU(tree,vector)
    find_NP_NR = NP_NR(tree,vector)
    find_CP_ADVP_IP = CP_ADVP_IP(tree,vector)
    find_NP_DNP_NP = NP_DNP_NP(tree,vector)
    find_ADVP_CS = ADVP_CS(tree,vector)
    find_DNP_NP_DEG = DNP_NP_DEG(tree,vector)
    #NP features
    find_NP_QP_DNP_NP = NP_QP_DNP_NP(tree,vector)
    find_NP_NP_PRN = NP_NP_PRN(tree,vector)
    find_NP_NR_CC_NR = NP_NR_CC_NR(tree,vector)
    find_NP_NP_CC_NP = NP_NP_CC_NP(tree,vector)

    return vector



In [4]:
def get_dependency_parsing(sentences):
    parsed_words = []
    for sentence in sentences:
        sentence = sentence.strip()  # 去除多余的空白字符
        if not sentence:
            continue
        doc_dependency = nlp_dependency(sentence)
        parsed_words.append(doc_dependency)

In [5]:

def get_feature_vector_dep(sentence,vector):
    doc_dependency = nlp_dependency(sentence)
    a = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','nsubj','我')
    b = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','advmod','将')
    c = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','nsubj','他')
    d = depTripleFuncLex(doc_dependency.sentences[0].words,'NN','det','该')
    e = depTripleFuncLex(doc_dependency.sentences[0].words,'NR','case','的')
    f = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','nsubj','他们')
    g = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','nsubj','她')
    h = depTripleFuncLex(doc_dependency.sentences[0].words,'他','case','的')
    i = depTripleFuncLex(doc_dependency.sentences[0].words,'NN','nmod:assmod','他')
    j = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','punct','。')
    k = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','advmod','但是')
    l = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','nsubj','你')
    m = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','advmod','如果')
    n = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','mark','的')
    o = depTripleFuncLex(doc_dependency.sentences[0].words,'NN','det','任何')
    p = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','case','因为')
    q = depTripleFuncLex(doc_dependency.sentences[0].words,'NR','cc','和')
    r = depTripleFuncLex(doc_dependency.sentences[0].words,'NN','det','那些')
    s = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','nsubj','它')
    t = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','dobj','它')
    if a:
        vector[14] = 1
    if b:
        vector[15] = 1
    if c:
        vector[16] = 1
    if d:
        vector[17] = 1
    if e:
        vector[18] = 1
    if f:
        vector[19] = 1
    if g:
        vector[20] = 1
    if h:
        vector[21] = 1
    if i:
        vector[22] = 1
    if j:
        vector[23] = 1
    if k:
        vector[24] = 1
    if l:
        vector[25] = 1
    if m:
        vector[26] = 1
    if n:
        vector[27] = 1
    if o:
        vector[28] = 1
    if p:
        vector[29] = 1
    if q:   
        vector[30] = 1
    if r:
        vector[31] = 1
    if s:
        vector[32] = 1
    if t:
        vector[33] = 1
    return vector


In [6]:
vector = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

In [7]:
# 确定运行环境，是cuda还是cpu
ENV = ""
if torch.cuda.is_available():
    ENV="cuda"
else:
    ENV="cpu"

In [8]:
# 原始模型和分词器的初始化
origin_model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt").to(ENV)
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer.src_lang = "en_XX"


In [9]:
decoder_forward_func = origin_model.model.decoder.layers[11].forward

random_number = random.uniform(-1, 1)
t = 1
lasttoken_pos = 0
last_token_state = torch.zeros(5, 1, 1024)


In [10]:

def new_decoder_activate(hidden_states, attention_mask = None, encoder_hidden_states = None, encoder_attention_mask = None,
                         layer_head_mask = None, cross_attn_layer_head_mask = None, past_key_value = None, output_attentions = False,
                         use_cache = True):
    global t
    global last_token_state
    if t % 4 == 0:
        random_number = random.uniform(-1, 1)
        modified_hidden_states = hidden_states
        modified_hidden_states += random_number
        modified_hidden_states = torch.nn.functional.gelu(modified_hidden_states)
    else:
        modified_hidden_states = hidden_states  # 加随机数 torch.nn.functional.gelu
    t+=1
    t%=4
    last_token_state = modified_hidden_states
    # 调用原始 decoder 层的 forward 方法
    output = decoder_forward_func(
        modified_hidden_states, attention_mask, encoder_hidden_states,
        encoder_attention_mask, layer_head_mask, cross_attn_layer_head_mask,
        past_key_value, output_attentions, use_cache
    )
    # final_output = output * 2  # 举例，对输出进行缩放
    return output

In [11]:
# 激活模型
if 'activated_model' in globals():
    del activated_model
    # 清理未使用的显存
    torch.cuda.empty_cache()
    # 强制垃圾回收
    gc.collect()
    


In [12]:
activated_model = deepcopy(origin_model)


In [13]:

activated_model.model.decoder.layers[11].forward = new_decoder_activate

In [14]:
cluster0_encodes = []
with open('.\\processed\\2024-09-16-13-39-51_clustered.json', 'r', encoding='utf-8') as file:
    cluster = json.load(file)
cluster0 = cluster['0']
for item in cluster0:
    tmp_encoded_input = tokenizer(item['en'], return_tensors="pt").to(ENV)
    cluster0_encodes.append(tmp_encoded_input)

In [15]:
def list1(l):
    sum =0
    for i in l:
        sum+=i
    return sum

def str1(s):
    sum = 0
    for c in s:
        sum+=int(c)
    return sum

def l2s(tmp):
    return ''.join(str(x) for x in tmp)

In [16]:
translated_sens=[]
cluster0_steer = torch.zeros(5, 1, 1024).to(ENV)
for index, encoded_input in enumerate(cluster0_encodes):
    i=10
    zh_tmp = []
    fvec_list_tmp = []
    better = 0
    last_token_hidden_states = torch.zeros(5, 1, 1024).to(ENV)
    while i > 0:
        t = 1
        generated_tokens = activated_model.generate(
            **encoded_input, 
            forced_bos_token_id=tokenizer.lang_code_to_id["zh_CN"]
        )
        translated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
        translated_text = translated_text.replace(" ", "")
        fvc_tmp = get_feature_vector(translated_text,vector)
        fvc_tmp = get_feature_vector_dep(translated_text,vector)
        vector = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
        if list1(fvc_tmp) < str1(cluster0[index]['fvec']):
            better+=1
            zh_tmp.append(translated_text)
            last_token_hidden_states += last_token_state
            fvec_list_tmp.append(l2s(fvc_tmp))
        i-=1
    if better > 0:
        cluster0_steer += last_token_hidden_states / better
    translated_sens.append({'en': cluster0[index]['en'], 'baseline': cluster0[index]['zh'], 'zh': zh_tmp , 'fvec': fvec_list_tmp})
    if (index + 1) % 10 == 0:
        # 当前时间
        current_time = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
        # 保存翻译结果
        with open(f'output/activated{current_time}.json', 'w', encoding='utf-8') as file:
            json.dump(translated_sens, file, ensure_ascii=False, indent=4, separators=(',', ':'))
        translated_sens=[]



In [20]:
print(cluster0_steer)

tensor([[[-941.3407, 3284.5481,  935.6570,  ..., 1477.7037, -390.3083,
          -451.2839]],

        [[-863.5672, 3005.5312,  898.7526,  ..., 1535.2255, -390.3169,
          -416.3390]],

        [[-358.0058, 2765.7063,  823.9050,  ..., 1495.9956, -547.4582,
          -400.3140]],

        [[ -83.2323, 2508.0881,  759.8983,  ..., 1456.4004, -645.4664,
          -293.4832]],

        [[ 191.5240, 2308.0579,  728.5186,  ..., 1427.7723, -796.5109,
          -269.1384]]], device='cuda:0')


In [21]:
with open('cluster0_steer.json', 'w', encoding='utf-8') as file:
    json.dump(cluster0_steer.tolist(), file, ensure_ascii=False, indent=4, separators=(',', ':'))

In [27]:

# 当前时间
current_time = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
with open(f'output/activated{current_time}.json', 'w', encoding='utf-8') as file:
    json.dump(translated_sens, file, ensure_ascii=False, indent=4, separators=(',', ':'))
translated_sens=[]